In [24]:
import os 
import pandas as pd 
from IPython.display import Markdown, HTML, display

In [25]:
from langchain.schema import HumanMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    openai_api_version="2024-04-01-preview",
    azure_deployment="SayliModelSQLite",
    azure_endpoint ="https://memi-openai-sayli.openai.azure.com/",
    api_key="17ddcb09784446f1bd2ef4ac4cd31cc4" 
)

In [26]:
message = HumanMessage(
    content="Translate this sentence from English "
    "to French and Spanish. I like red cars and "
    "blue houses, but my dog is yellow."
)

In [27]:
model.invoke([message])

AIMessage(content="Sure! Here’s the translation:\n\n**French:** J'aime les voitures rouges et les maisons bleues, mais mon chien est jaune.\n\n**Spanish:** Me gustan los coches rojos y las casas azules, pero mi perro es amarillo.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 31, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-Bmf3XUsuWXiqoWKivo0tosdPqaKwO', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'saf

In [28]:
df = pd.read_csv("F:/GenAI/Database Agent/Data/all-states-history.csv").fillna(value = 0)

In [29]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=model,df=df,verbose=True,allow_dangerous_code=True,handle_parsing_errors=True)

agent.invoke("how many rows are there?")

C:\Users\Aadipiya\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(




> Entering new AgentExecutor chain...
Thought: To determine the number of rows in the pandas DataFrame `df`, I can use the `shape` attribute, which returns a tuple where the first element is the number of rows and the second element is the number of columns.
Action: python_repl_ast
Action Input: df.shape[0]20780I now know the final answer.  
Final Answer: There are 20,780 rows in the DataFrame `df`.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 20,780 rows in the DataFrame `df`.'}

In [30]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = "How may patients were hospitalized during July 2020" 
"in Texas, and nationwide as the total of all states?"
"Use the hospitalizedIncrease column" 


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Thought: I need to set the pandas display options to show all columns, then retrieve the column names. After that, I will filter the data to find how many patients were hospitalized during July 2020. First, I will set the display options and retrieve the column names.

Action: python_repl_ast
Action Input: 
```python
import pandas as pd

# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)

# Get the column names
column_names = df.columns.tolist()
column_names
```['date', 'state', 'death', 'deathConfirmed', 'deathIncrease', 'deathProbable', 'hospitalized', 'hospitalizedCumulative', 'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative', 'inIcuCurrently', 'negative', 'negativeIncrease', 'negativeTestsAntibody', 'negativeTestsPeopleAntibody', 'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently', 'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore', 'positiveTests

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow may patients were hospitalized during July 2020\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got\nto the answer on a section that starts with: "\n\nExplanation:\n".\nIn the explanation, 

In [31]:
import os
from IPython.display import Markdown, HTML, display
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_openai import AzureChatOpenAI

In [32]:
from sqlalchemy import create_engine
import pandas as pd


df = pd.read_csv("F:/GenAI/Database Agent/Data/all-states-history.csv").fillna(value = 0)

In [33]:
import sqlite3

In [34]:
conn = sqlite3.connect("test.db")

In [35]:
# Path to your SQLite database file
database_file_path = "F:/GenAI/Database Agent/Scipts/test.db"

# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
engine = create_engine(f'sqlite:///{database_file_path}')
file_url = "F:/GenAI/Database Agent/Data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
df.to_sql(
    'all_states_history',
    con=engine,
    if_exists='replace',
    index=False
)

20780

In [36]:
MSSQL_AGENT_PREFIX = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [37]:
MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: 
SELECT TOP (10) [death]
FROM covidtracking 
WHERE state = 'TX' AND date LIKE '2020%'

Observation:
[(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state
is 'TX' and the date starts with '2020'. The query returned a list of tuples
with the number of deaths for each day in 2020. To answer the question,
I took the sum of all the deaths in the list, which is 27437.
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example

"""

In [41]:
llm = AzureChatOpenAI(
    openai_api_version="2024-04-01-preview",
    azure_deployment="SayliModelSQLite",
    azure_endpoint="https://memi-openai-sayli.openai.azure.com/",
    api_key="17ddcb09784446f1bd2ef4ac4cd31cc4",
    temperature=0, 
    max_tokens=500
)
db_path = "F:/GenAI/Database Agent/Scipts/test.db"
db = SQLDatabase.from_uri(f'sqlite:///{db_path}')
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [42]:
QUESTION = """How may patients were hospitalized during October 2020
in New York, and nationwide as the total of all states?
Use the hospitalizedIncrease column
"""

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

In [43]:
agent_executor_SQL.invoke(QUESTION)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: all_states_historyI have identified the table `all_states_history` which likely contains the relevant data for hospitalization statistics. Now, I will check the schema of this table to understand its structure and the available columns.

Action: sql_db_schema
Action Input: all_states_history
CREATE TABLE all_states_history (
	date TEXT, 
	state TEXT, 
	death FLOAT, 
	"deathConfirmed" FLOAT, 
	"deathIncrease" BIGINT, 
	"deathProbable" FLOAT, 
	hospitalized FLOAT, 
	"hospitalizedCumulative" FLOAT, 
	"hospitalizedCurrently" FLOAT, 
	"hospitalizedIncrease" BIGINT, 
	"inIcuCumulative" FLOAT, 
	"inIcuCurrently" FLOAT, 
	negative FLOAT, 
	"negativeIncrease" BIGINT, 
	"negativeTestsAntibody" FLOAT, 
	"negativeTestsPeopleAntibody" FLOAT, 
	"negativeTestsViral" FLOAT, 
	"onVentilatorCumulative" FLOAT, 
	"onVentilatorCurrently" FLOAT, 
	positive FLOAT, 
	"positiveCasesViral" FLOAT, 
	"positiveIncrease" BIGINT, 


{'input': 'How may patients were hospitalized during October 2020\nin New York, and nationwide as the total of all states?\nUse the hospitalizedIncrease column\n',
 'output': "In October 2020, there were 0 hospitalized patients recorded in New York, while nationwide, there were a total of 53,485 hospitalized patients.\n\nExplanation: I queried the `all_states_history` table to find the total number of hospitalized patients in New York during October 2020 by summing the `hospitalizedIncrease` column for that state and date range. The result was 0, indicating no recorded increases in hospitalizations for that month in New York. For the nationwide total, I summed the same column for all states during October 2020, which resulted in 53,485. \n\nThe queries used were:\n```sql\nSELECT SUM(hospitalizedIncrease) AS total_hospitalized_ny\nFROM all_states_history\nWHERE state = 'NY' AND date LIKE '2020-10%'\n\nSELECT SUM(hospitalizedIncrease) AS total_hospitalized_nationwide\nFROM all_states_his